In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array

import tensorflow as tf
import random as rn

In [0]:
# Setting random seeds
np.random.seed(42)
rn.seed(42)
tf.set_random_seed(42)

### Задание. 

Разархивировать данные в Colab и подключить директорию с датасетом. Смотреть предыдущий ноутбук

In [0]:
from google.colab import drive # монтируем гугл диск
drive.mount('/content/gdrive')

In [0]:
import os

os.chdir('/content/gdrive/My Drive/Colab Notebooks/Datasets/')  # меняем директорию по умолчанию, куда загружен датасет (у каждого своя)

# Дополнительно можно создать директорию прямо в Colab и разархивировать туда!
# !mkdir data  # создание директории
# !unzip -q data_classification.zip # разархивирование

### 1. Генератор данных в Keras

In [0]:
image_size = 224
batch_size = 10

data_generator = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )
# можно попробовать добавить и другие аугментации, не только zoom, флип, вращения и сдвиги, но этих будет достаточно
# image data generator будет автоматически в процессе тренировки генерировать бэтчи изображений с аугментациями
# https://keras.io/preprocessing/image/

train_generator = data_generator.flow_from_directory(
        '../input/flowers-recognition/flowers/flowers/',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

num_classes = len(train_generator.class_indices)

### 2. Предобученная модель ResNet-50 в качестве feature extractor

https://keras.io/applications/

In [1]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights="imagenet")) # include_top = False без полносвязных слоев, только сверточный feature extractor

model.add(Flatten()) 
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu')) # можно попробовать добавить l2 регуляризацию в dense слой
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu')) # можно попробовать добавить l2 регуляризацию в dense слой
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False # веса ResNet замораживаем

NameError: ignored

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Задание 1. Добавить кастомную метрику

Добавить кастомную метрику Macro F1 score для многоклассовой классификации

https://keras.io/metrics/

In [0]:
import keras.backend as K

def f1(y_true, y_pred): # реализация macro f1-score 
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [0]:
model.summary()

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=int(count/batch_size) + 1,
        epochs=10)

### 3. Визуализация результатов

In [0]:
from IPython.display import Image, display
import os, random

# набираем произвольный семпл изображений для инференса

img_locations = []

for d in os.listdir("../input/flowers-recognition/flowers/flowers/"):
    directory = "../input/flowers-recognition/flowers/flowers/" + d
    sample = [directory + '/' + s for s in random.sample(
        os.listdir(directory), int(random.random()*10))] 
    img_locations += sample

In [0]:
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    return preprocess_input(img_array)

In [0]:
random.shuffle(img_locations)
imgs = read_and_prep_images(img_locations)

predictions = model.predict_classes(imgs)

In [0]:
classes = dict((v,k) for k,v in train_generator.class_indices.items())

for img, prediction in zip(img_locations, predictions):
    display(Image(img))
    print(classes[prediction])

### 4. Использование вызываемых функций (Callbacks) в Keras

https://keras.io/callbacks/

ModelCheckpoint - настроить сохранение весов модели в папку после каждой эпохи

EarlyStopping - ранняя остановка для предотвращения переобучения

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("resnet50_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

In [0]:
model_final.fit_generator(generator= train_generator, steps_per_epoch=2, epochs=20, 
                          validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])

In [0]:
model_final.save_weights("resnet50_class.h5")

### Задание 2. Подключить callbacks

Визуализировать Tensorboard

https://medium.com/@tommytao_54597/use-tensorboard-in-google-colab-16b4bb9812a6

https://medium.com/@kuanhoong/how-to-use-tensorboard-with-google-colab-43f7cf061fe4

### Задание 3. Реализовать Learning rate range test и Cyclic Learning Rate

Согласно статье
https://medium.com/@psk.light/one-cycle-policy-cyclic-learning-rate-and-learning-rate-range-test-f90c1d4d58da